In [14]:
import numpy as np                   
import pandas as pd                  
import matplotlib.pyplot as plt

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
model = MultinomialNB()

In [15]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [16]:
data = pd.read_csv("Apple-Twitter-Sentiment-DFE.csv", sep=';', encoding='utf-8', header = 0)

In [17]:
headers = data.iloc[0]
data.columns = headers
data = data.iloc[1:]

In [18]:
filtered_data = pd.DataFrame()
filtered_data['sentiment'] = data['sentiment']
filtered_data['sentiment confidence'] = data['sentiment:confidence']
filtered_data['text'] = data['text']
text_data = pd.DataFrame({'text': filtered_data['text']})
text_data.shape

(3886, 1)

In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob

# Download NLTK stop words data
nltk.download('stopwords')

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove URLs, special characters, and punctuation
    text = ' '.join(word for word in text.split() if "http://" not in word and "https://" not in word and "@" not in word)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    
    # Fix misspelled words using TextBlob (you can customize this part)
    text = str(TextBlob(text).correct())
    
    # Stem words using NLTK's Porter Stemmer
    stemmer = PorterStemmer()
    text = ' '.join(stemmer.stem(word) for word in text.split())
    print(text)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marius/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
def create_vocabulary_list2(data):
    vocabulary = set()
    for text in data['text']:
        # Preprocess each text
        preprocessed_text = preprocess_text(text)
        
        # Tokenize and add to vocabulary
        words = preprocessed_text.split()
        vocabulary.update(words)
    
    return vocabulary

In [22]:
def create_vocabulary_list(data):
    vocabulary = set()
    for text in data['text']:
        words = str(text).lower().split()
        vocabulary.update(words)

    vocabulary = {word for word in vocabulary if "http://" not in word}
    vocabulary = {word for word in vocabulary if "https" not in word}
    vocabulary = {word for word in vocabulary if "@" not in word}
    return vocabulary

In [23]:
def create_document_term_matrix(data, vocabulary_list):

    document_word_counts = []
    
    for _, row in data.iterrows():
        document = str(row['text']).lower()
        word_counts = {word: document.count(word) for word in vocabulary_list}
        document_word_counts.append(word_counts)

    # Create a DataFrame from the list of dictionaries
    document_term_matrix = pd.DataFrame(document_word_counts)
    document_term_matrix.index = range(1, len(document_term_matrix) + 1)
    document_term_matrix = document_term_matrix.fillna(0)  # Fill NaN values with 0
    return document_term_matrix

In [24]:
def individual_create_document_term_matrix(individual_data, vocabulary_list):

    document_word_counts = []
    word_counts = {word: individual_data.count(word) for word in vocabulary_list}
    document_word_counts.append(word_counts)

    # Create a DataFrame from the list of dictionaries
    document_term_matrix = pd.DataFrame(document_word_counts)
    document_term_matrix.index = range(1, len(document_term_matrix) + 1)
    document_term_matrix = document_term_matrix.fillna(0)  # Fill NaN values with 0
    return document_term_matrix

In [25]:
def combine_two_data_frames(df1, df2):
    df2.reset_index(drop=True, inplace=True)
    df1.reset_index(drop=True, inplace=True)

    # Combine the two DataFrames horizontally
    combined_df = pd.concat([df1, df2], axis=1)
    combined_df = combined_df[combined_df['sentiment'] != "not_relevant"]

    # Reset the index of the DataFrame
    combined_df.reset_index(drop=True, inplace=True)
    return combined_df

In [26]:
def check_text_with_document_term_matrix(row_index):

    # Check if the specified row contains '1.0' and print the corresponding column names
    value_to_check = 1.0
    columns_with_1 = combined_df.columns[combined_df.iloc[row_index] == value_to_check].tolist()

    print(combined_df.loc[row_index, 'text'])
    print(combined_df.loc[row_index, 'sentiment'])
    print(columns_with_1)

In [27]:
vocabulary_list = create_vocabulary_list2(text_data)

#all:th 10 best steve job email
it all stock mind-flash crash today $all #all
cat chess cords. #applesnob.
agre #individualinvestor trade #appl #all, extend today' pullback good see
nobodi expect spanish inquisit #all
#all:5 rocket stock buy decemb gains: appl
top 3 #tablets. damn right!
cnbctv: #apple' margin better expected? #all
appl in. flash crash: need know #all
#all:thi present show make world' biggest teach compani
utf batteri 31% one second ago 29% utf
appl watch top search engin list best bearabl teach #all #watch #applewatch
best-design #phone #ape world, accord
it bought store..pretti good log match . can't wait instal it!
contact son yosemit is serious screw up. use much stabl past. #cloud #isn
#all thank non actual dumb twitter follow stock drop 3 point one minute. thank duties. #rumor
warn buy phone is unlock phone cannot use horizon network
john cattl prison iri 739 days, show abandon him. sign
thank extra check upper westsid store- appoint run almost 50 minut late?
#al

love &amp; earn ~200 mile #blackfriday sale! #freemoney #centsabl #cybermonday
to! auto correct shit gutta stop!!! chang shit right send
first log design co-found donald wayne. draw isaac newton read book appl tree
those** pick slack fuck boy
firm acquir bloomberg termin servic yet #all? wish #good search function.
mayb good product never better malbrook air .... fact
fuck maggoti
cours that' mother fuck
appl in. store log turn red world aid day #all
fuck kind gay shit print fuck ketosi
game*** birch realli tri tonight! it game night and
appl co tim cook come gay via
it u need get or fuck shit togeth let txt coutur
can't buy version let alon version actual like star war #stupid #lame #stalwart
perhap less malleabl metal shell #iphone6splus. #noticedsomecurv
malbrook pro 13 non-retina first started: via #malbrook #macbookpro #start #sister
malbrook pro in undoing: via #malbrook #macbookpro #start #sister #undo
pad mind undoing!: via #ipadmini #pad #malbrook #macbookpro #start #sister #u

plato launch thunderbolt 2 dock k resolut support [mac blow] #all
#e-pap ye #watch coolest piec #bearabl #teach yet, sorri via
final gave return 17' malbrook pro model. 15' retina 'out delivery'. they'll probabl announc one next month ;-)
appl intraday comment - updat new rang - premium user #all #stockact #trade #stock #met
appl lose ground goos educ tablet sale $all #all
appl comput in (all) rate buy price target $135 canaccord genuin #all
need mac realli slow late ?
appl - fibonacci technic level - intraday updat - $all #all
it enjoy introduct #connect launch event. amaz #detect leader room. #ti
$all #all green right now. hope stays.
it custom servic success &amp; fail &amp; exampl #custer #c #ce
spring offer have tireless bill custom twitch at&amp;t horizon [i blow] #all
affluent appl user out-spend andro user number realli mean? $all #all
new mean soviet joke 'whatev produce, turn tank': #russia seek #kalashnikov rival
stupid shit upgrad ruin god damn side
piec shit god admit man!

a: speak parables! new charg &amp; phone 6. want eat lake run place. #tl_chat
sound like we'd travel well togeth then! #tl_chat
&it; a: ship here- inspir phone 6+~ take incred photo! pin sharp!
pleas meet fellow weep #connect event! fellow #coach practition great meet girl #ti
bodi eye level store #statu #nice
appl watch track februari 2015 release, test underway [rumour] $all #all
think 100 reason is. fuck
it appl watch track februari 2015 release, test underway [rumour] $all #all
it bodi eye level store #statu #nice
follow unlik tuesday? mani unlik #follow? #socialcleanuptuesday mayb #antidrama
agree. 'get' explicitli impli free.
also utf dont even live kirk
#innov sector largest multipli effect come jobs! employ 13,000, creat 70,000 indirect jobs!
morgan stanley cell appl in., all stock still buy $all #all
there' movi tune match / cloud can't remove. actual trailer. annoy
silver finish: #applewatch place second even via #all
sir bull shit tell donna rain shit da fuck pay
give job ne

one good thing wait store 'appointment' clear back email into
product hunters: function (and whimsical) keyboard #i - forc #all
move asid #appl #goose, #alabama enter connect car space - forc #babi #good #all
messag work thank
get andro
can't got new phone /;
type chang words. get togeth
$all 17 analyst rate last 30 day 94% positive. median target: $130 (13% upside). #all
hey could make phone charg last month? sincerely, broke high school student want phone charg
fuck appl allow pp access camera phone lock fuck fuck
myth vs. reality-ful time #trade #stock #futur #fore #trader #e #all #pro #signal
ape emboli phone? middl finger? almost 2015, step
begin think product cap
phone 6 final came almost 3 week think i'll forget
#all:in email, steve job determin keep upon
need send phone cancel
th charg i'v buy mac gave wrong one #applesuport fix wrongdo give free one
fuck
liber protest corpor greed throw away fifti products. #shutitdown #ferguson
watch steve wozniak use #phone magic clock hotel

...

#whitepow #hatemus #hatespeech #icantbreath good look
use congress gave govern permiss spi you. #appl
had. defend it. understand internet.
cool 90 minut later yosemit x malbrook bad almost 10 year old comput still run strong #win
appl in. catch phone 6 supply, still lag demand #all
alway impress custom servic store squar one. lawton work cost time!
still suicid note templ recent page updat much longer wait
spot left 10 dec event w/
thank make life complic
#met final #all #putcallratio monday, decemb 8, 2014:
final #all #putcallratio monday, decemb 8, 2014 via hazel $$
thank 'do disturb' featur
offens thing lack trashcan emboli stop discrimin
it offens thing lack trashcan emboli stop discrimin
it final #all #putcallratio monday, decemb 8, 2014 via hazel $$
appl in. vs goos in: battl teach giant #all
it thank 'do disturb' featur
hope &amp; #i allow compact view rd parti keyboard like #swiftli #type #andros.
malbrook pro annoy ask window 8 computer. freezing, baggy, etc. much excellence.


it protest stage #vein protest store ny... me, anger misplaced? between agree.
it protest stage #vein protest store ny... me, anger misplaced? between agree.
it protest stage #vein protest store ny... me, anger misplaced? between agree.
protest show form violence, violenc destructive:
black (13% pop.) murder 1000 peopl per or. white (63% pop.)! polic deal with!
it protest stage #vein protest store ny... me, anger misplaced? between agree.
it protest stage #vein protest store ny... me, anger misplaced? between agree.
action help cause. #democratswarontruth
it protest stage #vein protest store ny... me, anger misplaced? between agree.
seem kill geniu bar reserv onlin via mass appl store even.
it protest stage #vein protest store ny... me, anger misplaced? between agree.
judg rule bank sue credit card breach; #pay valu proposit intensifi
still misplaced, compani power something.
game continu occupi clown thing. tri reliev was. drug sex etc
it protest stage #vein protest store ny... me, an

phone work without him card g (not wife) #utf
phone mad sensit brow, cold, die even lot batteri left. hot, shut well such
it protest stage #vein protest store ny... me, anger misplaced? between agree.
donna let italic
#code #all - sourc kit crash swift - get annoy now.
it protest stage #vein protest store ny... me, anger misplaced? between agree.
fuck
worst thing ever updat softwar computer. updat trash
yeah
told he' crazi
link bring back pre order thing
#meet #elevate, [#apple]' pick best #phone pp #year. (#monday, 8 #december)
chanc appl to. #horizon #fit go downhil fast. use happi plastic sob displays!
ain't steal appl
it protest stage #vein protest store ny... me, anger misplaced? between agree.
i'm play game
is u delet contact actual delet for fuck u
it is u delet contact actual delet for fuck u
it protest stage #vein protest store ny... me, anger misplaced? between agree.
phone die 5% feel betrayed.

it studio 45,000 ft. one outlet, 4 computers. need batteri futur now!!!!
dear wo

#applewatch $11 billion impact earn one year #all
moment tie sweet offici client #pad 1 clashes. &it;:(
appl in. doomed. $all drop 1.11%! current price 111.155. #all
#all:finally! brooklyn get appl
build massiv new #r&amp;d center japan, say prime minist
china remind appl need support privacy, secur product $all #all
appl open advanc r&amp;d center yokohama, japan #all
major news outlet seek permiss share steve jobs' upon lawsuit deposit video #all
face
one phone 5 speaker stop working, that' cool..send phone 6 forgiv
store come brooklyn
appl - fibonacci technic level - intraday updat - $all #all
it appl - fibonacci technic level - intraday updat - $all #all
4-inch #phone: five reason #appl make - forc #all
it appl - fibonacci technic level - intraday updat - $all #all
appl add new appl to channel of, scene, fusion, dailymot #all
suck
it suck
coutur appl to pp updat streamlin interface, video advertis #all
report: #appl want fashion luxuri expert help sell #applewatch #all
book pick op

#all:here' apple' idea d phone
andro charg year &amp; they'r still top tier. pleas keep up,
appl intraday comment - last updat - premium user #all #stockact #trade #stock #met
pleas ban 'after school' 'in oak' ape tunes/pp store. ape use anonym build &amp; threats.
watch it!!!!!
recruit luxuri execut watch. i'll make easi them: get cerebr wear them.
who'r talk
apple:dec 9 trade daili profit $3.48/s, weekli profit $6.26/ #all #stockact #trade #stock #met
ll
let go shopping!!!
it i'v tri turn on, what' problem here?
i'm appl support
know want none!
think upbraid #yosemite? think twice everyone. it #arm
media report say hire pro fashion retail industri - #wearabletechnolog fashion
it protest stage #vein protest store ny... me, anger misplaced? between agree.
proof use products. break mail show search file finder! nut kicks.
doe
realli search tool useless. nut kick
or minut 10 minut
tre
it protest stage #vein protest store ny... me, anger misplaced? between agree.
gave huge middl finger fa

In [28]:
document_term_matrix = create_document_term_matrix(text_data, vocabulary_list)

In [29]:
combined_df = combine_two_data_frames(filtered_data, document_term_matrix)

In [30]:
check_text_with_document_term_matrix(1864)

text    Playing with @apple #healthkit. Somebody needs...
text                                                    0
Name: 1864, dtype: object
1
['oper', 'play', '#healthkit.', 'pp', '//', 'ay', ':', '#healthkit', 'sign', 'show', '#he', 'ak', 'si', 'axi', 'j', 'ye', '#de', 'it.', 'per', 'the', 'de', 'appl', 'le', 'proper', 'hi', 'es', 'lab', 'co', '#infograph', 'f', 'abe', 'el', 'make', 'em', 'design', 'is', 'x', 'be', 'heal', 'so', 'els', 'wit', '6', ':/', 'health', 'pro', 'some', 'labels.', '#i', 'healthkit', '#in', 'graph', 'need', '#design', '#d', 'with', 'kit', 'graphic', 'ma']


In [31]:
X = combined_df.iloc[:, 3:]
y = combined_df['sentiment']

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.5)

In [33]:
model.fit(X_train,Y_train)

MultinomialNB()

In [34]:
y_pred = model.predict(X_test)
from sklearn.metrics import classification_report
classification = classification_report(Y_test,y_pred)
print(classification)

              precision    recall  f1-score   support

           1       0.63      0.76      0.69       605
           3       0.76      0.82      0.79      1075
           5       0.92      0.10      0.18       222

    accuracy                           0.71      1902
   macro avg       0.77      0.56      0.55      1902
weighted avg       0.74      0.71      0.69      1902



In [35]:
# Preprocess the individual tweet (replace 'individual_tweet' with your actual tweet text)
individual_tweet = "I'm really liking the new apple laptop!"
preprocessed_tweet = individual_tweet.lower()  # Convert to lowercase
preprocessed_tweet = [word for word in preprocessed_tweet.split() if "http://" not in word and "https://" not in word and "@" not in word]  # Remove URLs and special characters

individual_tweet_document_term_matrix = individual_create_document_term_matrix(preprocessed_tweet, vocabulary_list)

In [36]:
predicted_sentiment = model.predict(individual_tweet_document_term_matrix)
print("Predicted Sentiment:", predicted_sentiment[0])

Predicted Sentiment: 1
